In [4]:
import pandas as pd
import numpy as np
import configparser
from sqlalchemy import create_engine

In [2]:
# Extract dataframes from the database first
lo = pd.read_csv("location.csv")

In [3]:
pa = pd.read_csv("rpause.csv")

In [4]:
qu = pd.read_csv("question.csv")

In [5]:
sc = pd.read_csv("sentence.csv")

In [6]:
st = pd.read_csv("sentiment.csv")

In [7]:
valid = ['2018-09-09 07:11:55', '2018-09-09 09:52:17', '2018-09-09 13:11:44', '2018-09-09 17:56:57', '2018-09-09 22:25:51']

In [11]:
v = '2018-09-09 09:52:17'

In [9]:
config = configparser.ConfigParser()
config.read('config.ini')

def connect():
    engine = create_engine('mysql+mysqldb://{}:{}@{}'.format(config['AWS']['RDS_USERNAME'],config['AWS']['RDS_PASSWORD'], config['AWS']['RDS_HOST'] ))
    return engine

def extractFeatureTable(username, class_timestamp, table_name):
    db = connect()
    db.execute('use features')
    return(pd.read_sql('''select * from {} where `username` = '{}' and `class_timestamp` = '{}' '''.format(table_name,username,class_timestamp),con=db))

In [10]:
sentenceDF = extractFeatureTable('trang', '2018-09-09 09:52:17', "sentences")

ModuleNotFoundError: No module named 'MySQLdb'

In [15]:
!pip install MySQL-python

  Using cached https://files.pythonhosted.org/packages/a5/e9/51b544da85a36a68debe7a7091f068d802fc515a3a202652828c73453cad/MySQL-python-1.2.5.zip
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/private/var/folders/20/0wmrh3w10nj3flcng71vr5sr0000gp/T/pip-install-r9tlw94k/MySQL-python/setup.py", line 13, in <module>
        from setup_posix import get_config
      File "/private/var/folders/20/0wmrh3w10nj3flcng71vr5sr0000gp/T/pip-install-r9tlw94k/MySQL-python/setup_posix.py", line 2, in <module>
        from ConfigParser import SafeConfigParser
    ModuleNotFoundError: No module named 'ConfigParser'
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /private/var/folders/20/0wmrh3w10nj3flcng71vr5sr0000gp/T/pip-install-r9tlw94k/MySQL-python/


In [16]:
pip install mysqlclient


The following command must be run outside of the IPython shell:

    $ pip install mysqlclient

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [39]:
lo

,username,class_timestamp,subject,grade,date,video_location,audio_location,transcript_location
0,trang,9/7/18 18:19,Math,4,9/7/18 0:00,http://classroom-bucket.s3.amazonaws.com/video...,http://classroom-bucket.s3.amazonaws.com/audio...,NaN
1,trang,9/9/18 7:11,Science,10,8/20/18 0:00,http://classroom-bucket.s3.amazonaws.com/video...,http://classroom-bucket.s3.amazonaws.com/audio...,http://classroom-bucket.s3.amazonaws.com/trans...
2,trang,9/9/18 9:52,Math,6,8/21/18 0:00,http://classroom-bucket.s3.amazonaws.com/video...,http://classroom-bucket.s3.amazonaws.com/audio...,http://classroom-bucket.s3.amazonaws.com/trans...
3,trang,9/9/18 13:11,English,7,8/21/18 0:00,http://classroom-bucket.s3.amazonaws.com/video...,http://classroom-bucket.s3.amazonaws.com/audio...,http://classroom-bucket.s3.amazonaws.com/trans...
4,trang,9/9/18 17:56,Biology,9,8/27/18 0:00,http://classroom-bucket.s3.amazonaws.com/video...,http://classroom-bucket.s3.amazonaws.com/audio...,http://classroom-bucket.s3.amazonaws.com/trans...
5,trang,9/9/18 22:25,Math,9,7/9/18 0:00,http://classroom-bucket.s3.amazonaws.com/video...,http://classroom-bucket.s3.amazonaws.com/audio...,http://classroom-bucket.s3.amazonaws.com/trans...


In [15]:
def get_teacher(sentenceDF):
    """
    Helper function to decide who is the teacher
    The longer string is the teacher
    :param sentenceDF: pd df returned from process_nlp
    :return: list of teacher's words, speaker tag of teacher
    """
    spkDF1 = sentenceDF[sentenceDF['speaker_label'] == 'spk_1']
    spkDF0 = sentenceDF[sentenceDF['speaker_label'] == 'spk_0']
    spkStr1 = spkDF1['sentence'].str.cat(sep=' ')
    spkStr0 = spkDF0['sentence'].str.cat(sep=' ')

    if len(spkStr0) > len(spkStr1):
        teacherStr = spkStr0
        teacher = 'spk_0'
    else:
        teacherStr = spkStr1
        teacher = 'spk_1'
    return teacher

In [12]:
pa1 = pa[pa['class_timestamp'] == v]
qu1 = qu[qu['class_timestamp'] == v]
sc1 = sc[sc['class_timestamp'] == v]
st1 = st[st['class_timestamp'] == v]

In [13]:
pa1

,username,class_timestamp,content,end_time,response_duration,same_speaker,speaker_label,start_time,type
6,trang,2018-09-09 09:52:17,What ?,86.776,8.190,0.0,spk_1,86.606,whQuestion
7,trang,2018-09-09 09:52:17,What ?,111.966,23.092,0.0,spk_0,111.866,whQuestion
8,trang,2018-09-09 09:52:17,Okay ?,147.768,36.060,0.0,spk_0,147.458,ynQuestion
9,trang,2018-09-09 09:52:17,"Other strategies , right ?",329.410,0.710,1.0,spk_1,326.300,ynQuestion
10,trang,2018-09-09 09:52:17,i ?,339.310,2.790,1.0,spk_1,338.900,ynQuestion
11,trang,2018-09-09 09:52:17,What ?,367.260,65.460,1.0,spk_1,367.150,whQuestion
12,trang,2018-09-09 09:52:17,How do we get ?,735.951,2.790,0.0,spk_1,735.141,whQuestion
13,trang,2018-09-09 09:52:17,Why ?,749.951,26.300,1.0,spk_0,749.641,whQuestion
14,trang,2018-09-09 09:52:17,What ?,831.991,1.210,1.0,spk_0,831.841,whQuestion
15,trang,2018-09-09 09:52:17,Four ?,833.751,1.190,1.0,spk_0,833.201,ynQuestion


In [16]:
teacher = get_teacher(sc1)

In [19]:
pa2 = pa1[pa1['speaker_label'] == teacher]

In [20]:
pa2

,username,class_timestamp,content,end_time,response_duration,same_speaker,speaker_label,start_time,type
6,trang,2018-09-09 09:52:17,What ?,86.776,8.19,0.0,spk_1,86.606,whQuestion
9,trang,2018-09-09 09:52:17,"Other strategies , right ?",329.410,0.71,1.0,spk_1,326.300,ynQuestion
10,trang,2018-09-09 09:52:17,i ?,339.310,2.79,1.0,spk_1,338.900,ynQuestion
11,trang,2018-09-09 09:52:17,What ?,367.260,65.46,1.0,spk_1,367.150,whQuestion
12,trang,2018-09-09 09:52:17,How do we get ?,735.951,2.79,0.0,spk_1,735.141,whQuestion
16,trang,2018-09-09 09:52:17,Right ?,853.122,0.21,1.0,spk_1,852.872,ynQuestion


In [24]:
table = pd.pivot_table(pa2, values='response_duration', index=['type'],columns=['same_speaker'], aggfunc=np.mean)

In [34]:
question_table = table.reset_index('type')

In [35]:
question_table

same_speaker,type,0.0,1.0
0,whQuestion,5.49,65.460000
1,ynQuestion,NaN,1.236667


In [39]:
question_table[question_table['type']=='ynQuestion'].reset_index().loc[0,1]

1.2366666666666788

In [28]:
['why_student'] = question_table[question_table['type']=='whQuestion']
['yn_student'] = table.loc['ynQuestion', 0]
['why_teacher'] = table.loc['whQuestion', 1]
['yn_teacher'] = table.loc['ynQuestion', 1]

5.48999999999998

In [36]:
# Pause analysis
def pause(pa, qu, sc, st, valid):
    lst_dict = []
    for v in valid:
        pa1 = pa[pa['class_timestamp'] == v]
        qu1 = qu[qu['class_timestamp'] == v]
        sc1 = sc[sc['class_timestamp'] == v]
        st1 = st[st['class_timestamp'] == v]
        
        d = {
            'SPEAKER': {'spk_0': 0, 'spk_1': 0, 'teacher': None},
            'TYPE': {'whQuestion': 0, 'ynQuestion': 0},
            'LENGTH': 0,
            'SENTIMENT': 0,
            'PAUSE': 0,
            'RESPONSE SPEAKER': {1: 0, 0: 0},
            'RESPONSE TYPE': {'whQuestion':0, 'Statement':0, 'System':0, 'ynQuestion':0, 'Clarify':0,
               'nAnswer':0, 'Greet':0, 'Reject':0, 'yAnswer':0, 'Accept':0, 'Continuer':0,
               'Other':0, 'Emphasis':0, 'Bye':0},
            'RESPONSE DURATION': 0,
            'RESPONSE SENTIMENT': 0,
            'skip': 0
        }
        
        print(v)
        print()
        d['SPEAKER']['teacher'] = get_teacher(sc1)
        for q in range(pa1.shape[0]):
            print('SPEAKER', pa1.iloc[q, 6])
            d['SPEAKER'][pa1.iloc[q, 6]] += 1

            print('TYPE:', pa1.iloc[q, 8])
            d['TYPE'][pa1.iloc[q, 8]] += 1

            print('LENGTH:', pa1.iloc[q, 3] - pa1.iloc[q, 7])
            d['LENGTH'] += (pa1.iloc[q, 3] - pa1.iloc[q, 7]) / pa1.shape[0]

            sentence_index = sc1[(sc1['end_time'] == pa1.iloc[q, 3]) & (sc1['start_time'] == pa1.iloc[q, 7])].iloc[0,2]
            print('SENTIMENT:',  st1[st1['sentence_index'] == sentence_index].iloc[0,3])
            d['SENTIMENT'] += (st1[st1['sentence_index'] == sentence_index].iloc[0,3]) / pa1.shape[0]
            
            try:
                print('PAUSE:', sc1[sc1['sentence_index'] == sentence_index+1].iloc[0,5] - pa1.iloc[q, 3])
                d['PAUSE'] += (sc1[sc1['sentence_index'] == sentence_index+1].iloc[0,5] - pa1.iloc[q, 3]) / pa1.shape[0]

                print('RESPONSE SPEAKER:', pa1.iloc[q, 5])
                d['RESPONSE SPEAKER'][pa1.iloc[q, 5]] += 1

                print('RESPONSE TYPE:', qu1[qu1['sentence_index'] == sentence_index+1].iloc[0,3])
                d['RESPONSE TYPE'][qu1[qu1['sentence_index'] == sentence_index+1].iloc[0,3]] += 1

                print('RESPONSE DURATION:', sc1[sc1['sentence_index'] == sentence_index+1].iloc[0,6] - sc1[sc1['sentence_index'] == sentence_index+1].iloc[0,5])
                d['RESPONSE DURATION'] += (sc1[sc1['sentence_index'] == sentence_index+1].iloc[0,6] - sc1[sc1['sentence_index'] == sentence_index+1].iloc[0,5])/ pa1.shape[0]

                print('RESPONSE SENTIMENT:',  st1[st1['sentence_index'] == sentence_index+1].iloc[0,3])
                d['RESPONSE SENTIMENT'] += (st1[st1['sentence_index'] == sentence_index+1].iloc[0,3]) / pa1.shape[0]
            except:
                d['skip'] += 1
            
            print()
        lst_dict.append(d)
    
    return lst_dict

In [37]:
lst_dict = pause(pa, qu, sc, st, valid)

2018-09-09 07:11:55

SPEAKER spk_0
TYPE: whQuestion
LENGTH: 3.6399999999999864
SENTIMENT: 0.5
PAUSE: 0.010000000000005116
RESPONSE SPEAKER: 1.0
RESPONSE TYPE: Clarify
RESPONSE DURATION: 2.8700000000000045
RESPONSE SENTIMENT: 0.0

SPEAKER spk_0
TYPE: ynQuestion
LENGTH: 0.5999999999999943
SENTIMENT: 0.5
PAUSE: 0.05000000000001137
RESPONSE SPEAKER: 1.0
RESPONSE TYPE: Clarify
RESPONSE DURATION: 211.06
RESPONSE SENTIMENT: 0.24599267399267397

SPEAKER spk_0
TYPE: whQuestion
LENGTH: 2.769999999999982
SENTIMENT: 0.375
PAUSE: 0.38999999999998636
RESPONSE SPEAKER: 1.0
RESPONSE TYPE: Clarify
RESPONSE DURATION: 16.210000000000036
RESPONSE SENTIMENT: -0.0805555555555556

2018-09-09 09:52:17

SPEAKER spk_1
TYPE: whQuestion
LENGTH: 0.1700000000000017
SENTIMENT: 0.0
PAUSE: 8.189999999999998
RESPONSE SPEAKER: 0.0
RESPONSE TYPE: Statement
RESPONSE DURATION: 0.20999999999999375
RESPONSE SENTIMENT: 0.2

SPEAKER spk_0
TYPE: whQuestion
LENGTH: 0.10000000000000853
SENTIMENT: 0.0
PAUSE: 23.091999999999985
RES

RESPONSE TYPE: Statement
RESPONSE DURATION: 0.2699999999999818
RESPONSE SENTIMENT: 0.0

SPEAKER spk_0
TYPE: whQuestion
LENGTH: 0.5699999999999932
SENTIMENT: 0.0
PAUSE: 0.0
RESPONSE SPEAKER: 1.0
RESPONSE TYPE: ynQuestion
RESPONSE DURATION: 1.2700000000000387
RESPONSE SENTIMENT: 0.0

SPEAKER spk_0
TYPE: ynQuestion
LENGTH: 1.2700000000000387
SENTIMENT: 0.0
PAUSE: 1.7899999999999636
RESPONSE SPEAKER: 0.0
RESPONSE TYPE: Accept
RESPONSE DURATION: 0.6100000000000136
RESPONSE SENTIMENT: 0.5

SPEAKER spk_0
TYPE: whQuestion
LENGTH: 1.7100000000000364
SENTIMENT: 0.0
PAUSE: 1.420000000000016
RESPONSE SPEAKER: 0.0
RESPONSE TYPE: ynQuestion
RESPONSE DURATION: 0.6899999999999977
RESPONSE SENTIMENT: 0.0

SPEAKER spk_1
TYPE: ynQuestion
LENGTH: 0.6899999999999977
SENTIMENT: 0.0
PAUSE: 0.009999999999990905
RESPONSE SPEAKER: 0.0
RESPONSE TYPE: whQuestion
RESPONSE DURATION: 1.5500000000000114
RESPONSE SENTIMENT: 0.0

SPEAKER spk_0
TYPE: whQuestion
LENGTH: 1.5500000000000114
SENTIMENT: 0.0
PAUSE: 0.03999999

RESPONSE TYPE: whQuestion
RESPONSE DURATION: 0.44999999999998863
RESPONSE SENTIMENT: 0.0

SPEAKER spk_1
TYPE: whQuestion
LENGTH: 0.44999999999998863
SENTIMENT: 0.0
PAUSE: 0.0
RESPONSE SPEAKER: 1.0
RESPONSE TYPE: Reject
RESPONSE DURATION: 7.640000000000043
RESPONSE SENTIMENT: 0.0863095238095238

SPEAKER spk_1
TYPE: whQuestion
LENGTH: 3.3100000000000023
SENTIMENT: 0.5
PAUSE: 1.490000000000009
RESPONSE SPEAKER: 1.0
RESPONSE TYPE: Statement
RESPONSE DURATION: 0.5099999999999909
RESPONSE SENTIMENT: 0.0

SPEAKER spk_1
TYPE: whQuestion
LENGTH: 4.710000000000036
SENTIMENT: 0.0
PAUSE: 0.67999999999995
RESPONSE SPEAKER: 1.0
RESPONSE TYPE: whQuestion
RESPONSE DURATION: 11.069999999999993
RESPONSE SENTIMENT: 0.0

SPEAKER spk_1
TYPE: whQuestion
LENGTH: 11.069999999999993
SENTIMENT: 0.0
PAUSE: 0.7900000000000773
RESPONSE SPEAKER: 1.0
RESPONSE TYPE: Statement
RESPONSE DURATION: 0.5099999999999909
RESPONSE SENTIMENT: 0.0

SPEAKER spk_0
TYPE: whQuestion
LENGTH: 0.6100000000000136
SENTIMENT: 0.0
PAUSE: 

In [31]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [38]:
lst_dict

[{'SPEAKER': {'spk_0': 3, 'spk_1': 0, 'teacher': 'spk_0'},
  'TYPE': {'whQuestion': 2, 'ynQuestion': 1},
  'LENGTH': 2.336666666666654,
  'SENTIMENT': 0.4583333333333333,
  'PAUSE': 0.15000000000000094,
  'RESPONSE SPEAKER': {1: 3, 0: 0},
  'RESPONSE TYPE': {'whQuestion': 0,
   'Statement': 0,
   'System': 0,
   'ynQuestion': 0,
   'Clarify': 3,
   'nAnswer': 0,
   'Greet': 0,
   'Reject': 0,
   'yAnswer': 0,
   'Accept': 0,
   'Continuer': 0,
   'Other': 0,
   'Emphasis': 0,
   'Bye': 0},
  'RESPONSE DURATION': 76.71333333333335,
  'RESPONSE SENTIMENT': 0.05514570614570613,
  'skip': 0},
 {'SPEAKER': {'spk_0': 5, 'spk_1': 6, 'teacher': 'spk_1'},
  'TYPE': {'whQuestion': 6, 'ynQuestion': 5},
  'LENGTH': 0.5709090909091012,
  'SENTIMENT': 0.07873376623376627,
  'PAUSE': 15.272909090909096,
  'RESPONSE SPEAKER': {1: 7, 0: 4},
  'RESPONSE TYPE': {'whQuestion': 0,
   'Statement': 7,
   'System': 1,
   'ynQuestion': 1,
   'Clarify': 0,
   'nAnswer': 0,
   'Greet': 0,
   'Reject': 0,
   'yAn